<a href="https://colab.research.google.com/github/AndreBeckPires/LLM-to-create-persona/blob/master/LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers accelerate bitsandbytes torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig
import os

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

In [ ]:
#ID do Modelo Original
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

#Caminho no seu Google Drive
model_save_path = "/content/drive/MyDrive/meus_modelos/mistral-7b-instruct"

In [ ]:
if os.path.exists(model_save_path):
    print(f"Carregando modelo do Google Drive... ({model_save_path})")
    tokenizer = AutoTokenizer.from_pretrained(model_save_path)
    model = AutoModelForCausalLM.from_pretrained(
        model_save_path,
        quantization_config=quantization_config,
        device_map="auto"
    )
else:
    print(f"Modelo não encontrado no Drive. Baixando de '{model_id}'...")
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        quantization_config=quantization_config,
        device_map="auto"
    )

    print(f"Salvando modelo no Google Drive para a próxima vez...")
    # Cria a pasta se não existir
    os.makedirs(model_save_path, exist_ok=True)
    model.save_pretrained(model_save_path)
    tokenizer.save_pretrained(model_save_path)

print("Modelo carregado com sucesso na GPU!")

Modelo não encontrado no Drive. Baixando de 'mistralai/Mistral-7B-Instruct-v0.2'...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Salvando modelo no Google Drive para a próxima vez...
Modelo carregado com sucesso na GPU!


In [57]:
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer
)

Device set to use cuda:0


In [61]:
instrucao_sistema_melhorada = (
    "Você é um chatbot especialista em marketing, falando em Português brasileiro e agindo como um ser humano."
    "Sua função é ajudar o usuário a construir uma persona completa, detalhe por detalhe. "
    "Os detalhes necessários são: nome, idade, gostos, hobbies, poder financeiro,região de nascimento e profissão."
    "Siga estas regras estritamente:"
    "1. Comece se apresentando e fazendo a primeira pergunta: 'Descreva sua persona' Espere o resultado responder"
    "2. Após o usuário responder, análise a resposta para abstrair detalhes, continue fazendo perguntas para obter os detalhes que faltam (hobbies, poder financeiro, etc.) sempre de forma a parecer que você é um humano. Não pergunte tudo de uma vez."
    "3. Seja amigável e conciso, utilizando poucas palavras e fazendo perguntas/sugestões diretas, e continue guiando o usuário passo a passo até ter coletado todos os detalhes da lista, mas fale como se você fosse um ser humano."
    "4. Quando você tiver todas as informações, e SOMENTE NESSE MOMENTO, você deve parar de perguntar e apresentar um resumo completo, começando EXATAMENTE com a frase: 'Todos detalhes da persona:' Não use essa frase em nenhum outro contexto"
)

In [62]:
# histórico começa com a instrução de sistema
historico_conversa = [
    {"role": "system", "content": instrucao_sistema_melhorada}
]
print("Chatbot Tutor pronto! Vá para a próxima célula para começar.")

Chatbot Tutor pronto! Vá para a próxima célula para começar.


In [64]:
print("Para começar, apenas diga 'oi' ou 'vamos começar'.")
print("Digite 'sair' a qualquer momento para terminar.")
print("-" * 30)

while True:
    prompt_usuario = input("Você: ")
    if prompt_usuario.lower() == 'sair':
        break

    # Adiciona a mensagem do usuário ao histórico
    historico_conversa.append({"role": "user", "content": prompt_usuario})

    # 1. Formatar o histórico para o modelo
    # O Mistral espera um template específico ([INST]...[/INST])
    # O tokenizer.apply_chat_template faz isso para nós.
    prompt_formatado = tokenizer.apply_chat_template(
        historico_conversa,
        tokenize=False,
        add_generation_prompt=True  # Adiciona o [/INST] para o bot responder
    )

    # 2. Gerar a resposta
    # max_new_tokens limita o tamanho da resposta
    # O 'pipe' retorna a conversa inteira, por isso precisamos limpá-la
    saida = pipe(
        prompt_formatado,
        max_new_tokens=256,
        do_sample=True,
        temperature=0.5,
        top_k=50,
        top_p=0.95
    )

    # 3. Limpar a resposta
    # A saída inclui o prompt, então pegamos apenas o texto novo
    resposta_completa = saida[0]['generated_text']
    resposta_bot = resposta_completa[len(prompt_formatado):].strip()

    print(f"Bot: {resposta_bot}")

    # 4. Adicionar a resposta do bot ao histórico
    historico_conversa.append({"role": "assistant", "content": resposta_bot})

    # (Opcional) Encerrar o quiz se ele disser "Correto!"
    if "Todos detalhes da persona:" in resposta_bot:
        print("-" * 30)
        print("Chat encerrado! Obrigado")
        break

Para começar, apenas diga 'oi' ou 'vamos começar'.
Digite 'sair' a qualquer momento para terminar.
------------------------------
Você: sair
